<a href="https://www.kaggle.com/code/meshkatshb/logging-zero-to-mastery?scriptVersionId=217907239" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Logging Zero To Mastery
In this tutorial we examine the logging package, how to use it, and when to use it.

# 1. Basic Logging Example
As you can see below, there are 5 types of loggings:
- `DEBUG (Level 10)`: Detailed information, typically of interest only when diagnosing problems.
- `INFO (Level 20)`: Confirmation that things are working as expected.
- `WARNING (Level 30)`: An indication that something unexpected happened or indicative of some problem in the near future.
- `ERROR (Level 40)`: Due to a more serious problem, the software has not been able to perform some function.
- `CRITICAL (Level 50)`: A very serious error, indicating that the program itself may be unable to continue running.

In [1]:
import logging

logging.basicConfig(level=logging.DEBUG)
logging.debug('This is a debug message')
logging.info('This is an info message')
logging.warning('This is a warning message')
logging.error('This is an error message')
logging.critical('This is a critical message')


# 2. Larger Applications
Let's say that we have large application, and we need to config it ourselves. So, instead of using `.basicConfig`, we can create our own config in `.json` format and pass it to the logging:

In [2]:
import logging.config

config = {
    'version': 1,
    'formatters': {
        'simple': {
            'format': '%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        },
    },
    'handlers': {
        'console': {
            'class': 'logging.StreamHandler',
            'formatter': 'simple',
            'level': 'DEBUG',
        },
    },
    'root': {
        'handlers': ['console'],
        'level': 'DEBUG',
    },
}

logging.config.dictConfig(config)
logging.debug('This is a debug message')


2025-01-16 11:56:08,110 - root - DEBUG - This is a debug message


# 3. Logging to a File
The best way to keep track of your logs and changes in your application is to save your logs. With logging, you can simply save your logs in a file:

In [3]:
logging.basicConfig(filename='./logs/app.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s')
logging.warning('This will get logged to a file')

2025-01-16 11:56:08,137 - root - WARNING - This will get logged to a file


### Note:
If you want to append to your existing file, you can use the `filemode='a'`, so it won't overwrite your previous logs. For example:

In [4]:
logging.basicConfig(filename='app.log', filemode='a', format='%(name)s - %(levelname)s - %(message)s')
logging.warning('This will get logged to a file')

2025-01-16 11:56:08,162 - root - WARNING - This will get logged to a file


### Note 2:
And if you want to save the timestamps in your format, you can add `%(asctime)s` to your format:

In [5]:
logging.basicConfig(filename='app.log', filemode='a', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logging.warning('This will get logged to a file')

2025-01-16 11:56:08,188 - root - WARNING - This will get logged to a file


### Note 3:
The `filename=` can't make directories for you, so you should create the directories first, and then it will create the file for you when you set the `filemode='w'` or it will append to your existing file with `filemode='a'`.

# 4. Logger Object
When your application is vast and very large, you may need different workers or logger objects for your application in order to keep track of what's happening in your application since the different places for logs should do different kinds of tasks. So, for more understanding, I will provide you with some examples:

In [6]:
logger = logging.getLogger('my_logger')
logger2 = logging.getLogger('my_logger2')

logger.setLevel(logging.DEBUG)
logger2.setLevel(logging.WARNING)

ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)

logger.addHandler(ch)
logger.debug('This is a debug message')
logger.warning('This is a warning message')

logger2.debug('This is a debug message by logger 2')
logger2.warning('This is a warning message by logger 2')

2025-01-16 11:56:08,225 - my_logger - DEBUG - This is a debug message
2025-01-16 11:56:08,225 - my_logger - DEBUG - This is a debug message
2025-01-16 11:56:08,228 - my_logger - WARNING - This is a warning message
2025-01-16 11:56:08,228 - my_logger - WARNING - This is a warning message
2025-01-16 11:56:08,229 - my_logger2 - WARNING - This is a warning message by logger 2


### Note:
As you can see, I set the `logger2` level to warning. Although I called the `logger2.debug`, but in the output we can't see the `my_logger2` object having any `DEBUG` message. This is only because of the priority of levels. In the [basic examples](#1-basic-logging-example), you can see the priority and levels of logs. Level 50 being higher priority and level 10 being the lowest priority. When using the higher priority, it will ignore the lower priority and won't show the logs below that level. So, keep that in mind.

# 5. Rotating Log Files
As you notice, while appending to your previous file, your log file can get larger and larger by time (even in TeraBytes), so to prevent storage problems, we can limit the log file. With `RotatingFileHandler` we can set maximum bytes in each file by setting the `maxBytes` and set how many back files it should store in `backupCount` attribute:

In [7]:
from logging.handlers import RotatingFileHandler

handler = RotatingFileHandler('app.log', maxBytes=2000, backupCount=5)
logger.addHandler(handler)

logger.debug('This is a sample debug message')


2025-01-16 11:56:08,269 - my_logger - DEBUG - This is a sample debug message
2025-01-16 11:56:08,269 - my_logger - DEBUG - This is a sample debug message


# 6. Logging Exceptions
One of the most useful things in python is to catch any exception when it occurs so that your application/program won't crash. In order to do so, you just use try-catch phrase as you may know. In this part, I show you that instead of simply printing the exception message or error, you can use logging to demonstrate the exception that occurred with `logger.exception`.

In [8]:
try:
    1 / 0
except ZeroDivisionError:
    logger.exception("Exception occurred")

2025-01-16 11:56:08,298 - my_logger - ERROR - Exception occurred
Traceback (most recent call last):
  File "/tmp/ipykernel_18/1521490165.py", line 2, in <module>
    1 / 0
ZeroDivisionError: division by zero
2025-01-16 11:56:08,298 - my_logger - ERROR - Exception occurred
Traceback (most recent call last):
  File "/tmp/ipykernel_18/1521490165.py", line 2, in <module>
    1 / 0
ZeroDivisionError: division by zero


# 7. Custom Level Logging
As I previously discussed in [this note](#note) and in [introduction](#1-basic-logging-example), we have different levels of logging:
- `DEBUG` = LEVEL 10
- `INFO` = LEVEL 20
- `WARNING` = LEVEL 30
- `ERROR` = LEVEL 40
- `CRITICAL` = LEVEL 50

You can define your own level of priority and add your custom label to the logging. To do so, you can simply do as below:

In [9]:
CUSTOM_LEVEL = 25
logging.addLevelName(CUSTOM_LEVEL, 'CUSTOM')

def custom(self, message, *args, **kwargs):
    if self.isEnabledFor(CUSTOM_LEVEL):
        self._log(CUSTOM_LEVEL, message, args, **kwargs)

logging.Logger.custom = custom
logger.custom('This is a custom log message')


2025-01-16 11:56:08,330 - my_logger - CUSTOM - This is a custom log message
2025-01-16 11:56:08,330 - my_logger - CUSTOM - This is a custom log message
